In [1]:
using MPSGE_MP, JuMP

In [9]:
M = MPSGEModel()

@parameter(M, tax, 0)
@parameter(M, σ, .5)
@parameter(M, id, 120)

@sectors(M,begin
    X
    Y
    W
end)

@commodities(M,begin
    PX
    PY
    PL
    PK
    PW
end)

@consumer(M, CONS)

add_production!(M,
    X,
    ScalarNest(:t; elasticity = 0, children = 
        [ScalarOutput(PX, id)]
    ),
    ScalarNest(:s; elasticity = σ, children = [
        ScalarInput(PY,20),
        ScalarNest(:va; elasticity = 1, children = [
            ScalarInput(PL, 40; taxes = [Tax(CONS, tax)]),
            ScalarInput(PK, 60; taxes = [Tax(CONS, tax)])
            ])
        ]
        ) 
)

add_production!(M,
    Y,
    ScalarNest(:t; elasticity = 0, children = [
        ScalarOutput(PY, 120),
    ]),
    ScalarNest(:s; elasticity = .75, children = [
        ScalarInput(PX,20),
        ScalarNest(:va; elasticity = 1, children = [
            ScalarInput(PL, 60),
            ScalarInput(PK, 40)
        ])
    ])
)


add_production!(M,
    W,
    ScalarNest(:t; elasticity = 0, children = [
        ScalarOutput(PW, 200)
    ]),
    ScalarNest(:s, elasticity = 1, children = [
        ScalarInput(PX, 100),
        ScalarInput(PY, 100),
    ])
)

add_demand!(M,
    CONS,
    [ScalarDem(PW, 200)],
    [
        ScalarEndowment(PL,100),
        ScalarEndowment(PK,100)
    ]
)

jm = build!(M);


# Benchmark

In [10]:
solve!(M; :cumulative_iteration_limit => 0)
df_benchmark = generate_report(M)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_6F68.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 9

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (f[    4])

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.0,0.0
2,W,1.0,0.0
3,Y,1.0,0.0
4,PL,1.0,0.0
5,PW,1.0,0.0
6,PK,1.0,0.0
7,PY,1.0,0.0
8,PX,1.0,0.0
9,CONS,200.0,0.0


# Counterfactual tax = .5

In [11]:
fix(PW,1)

set_value!(tax,.5)

solve!(M; :cumulative_iteration_limit => 10_000)
df = generate_report(M)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_715D.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             6.9173e+01             0.0e+00 (f[    6])
    1     1     0     8 2.9618e+00  1.0e+00    0.0e+00 (f[   11])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 2.9618e+00           I 0.0e+00 1.9e+00 (f[   11])
    1     1     3     3 2.4397e-02  1.0e+00 SO 0.0e+00 1.7e-02 (f[    7])
    2     1     4     4 2.5675e-06  1.0e+00 SO 0.0e+00 2.1e-06 (f[    7])
    3     1     5     5 9.9539e-14  1.0e+00 SO 0.0e+00 7.5e-14 (f[   12])

Major Iterations. . . . 3
Minor Iterations. . . . 3
Restarts. . . . . . . . 0
Crash Iteratio

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,0.861867,-3.55271e-14
2,W,0.983827,-4.26326e-14
3,Y,1.10915,7.10543e-15
4,PL,0.841191,-1.42109e-14
5,PW,1.0,8.52651e-14
6,PK,0.787062,1.42109e-14
7,PY,0.868686,2.84217e-14
8,PX,1.15116,-7.4607e-14
9,CONS,196.765,0.0


# Counterfactual tax = .1

In [12]:
fix(PW,1)

set_value!(tax,.1)

solve!(M; :cumulative_iteration_limit => 10_000)
df = generate_report(M)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_7371.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.4094e+01             0.0e+00 (f[    6])
    1     1     0     8 1.5012e-01  1.0e+00    0.0e+00 (f[   11])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.5012e-01           I 0.0e+00 1.1e-01 (f[   11])
    1     1     3     3 6.2994e-05  1.0e+00 SO 0.0e+00 4.2e-05 (f[    7])
    2     1     4     4 1.7039e-11  1.0e+00 SO 0.0e+00 1.4e-11 (f[    7])

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 4
Gradient E

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,0.968548,7.52287e-12
2,W,0.999102,2.98428e-13
3,Y,1.02732,6.84253e-12
4,PL,0.960787,1.90425e-12
5,PW,1.0,-2.7768e-11
6,PK,0.945827,1.34861e-11
7,PY,0.967574,-8.3844e-13
8,PX,1.03351,-9.9476e-14
9,CONS,199.82,5.11591e-13


In [14]:
fix(PW,1)

set_value!(σ,3)
set_value!(σ,100)

solve!(M; :cumulative_iteration_limit => 10_000)
df = generate_report(M)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_95B1.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.2503e+02             0.0e+00 (f[   10])
    1     4     0     8 4.3846e+01  4.1e-01    0.0e+00 (f[    7])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     5     2 4.3846e+01           I 0.0e+00 3.1e+01 (f[    7])
    1     1     6     3 5.0960e+00  1.0e+00 SM 0.0e+00 4.4e+00 (f[   10])
    2     1     7     4 1.8908e-02  1.0e+00 SO 0.0e+00 1.4e-02 (f[    7])
    3     1     8     5 1.8620e-05  1.0e+00 SO 0.0e+00 1.5e-05 (f[   10])
    4     1     9     6 4.8759e-12  1.0e+00 SO 0.0e+00 3.8e-12 (f[   10])

Major Iterations. 

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.09567,6.66134e-13
2,W,0.981912,1.42109e-14
3,Y,1.72391,-4.61853e-14
4,PL,1.11803,3.41061e-13
5,PW,1.0,-7.10543e-13
6,PK,0.822616,1.20792e-12
7,PY,0.992062,-1.44951e-12
8,PX,1.008,7.10543e-15
9,CONS,196.382,-8.52651e-14
